# Contents of This Note Book:
* Importing Libraries and Data Set
* Normalizing Data Set (aka Preprocessing)
* Code for past & future values as inputs & outputs
* Model Creation & Training
* Model Prediction & Accuracy Check
* TF Model to TF Lite Conversion
* TF Lite Interpreter
* TF Lite Model Prediction
* Comparison of TF, TF Lite & Orignal Values
* *** Created by Zaheer Ahmed ***


**Importing Libraries**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense , Flatten
import tensorflow as tf
print(tf.__version__)

**Importing Data Set**

In [ ]:
# uploading training data and test data files
# these files can be uploaded manually without writing this code ( by files tab
# on the left side )

from google.colab import files
data = files.upload()
test_data = files.upload()

# import dataset from data.csv file
dataset = pd.read_csv('data.csv')

# read values from all rows and temperature column
training_set = dataset.iloc[:,0:1].values

**Preprocessing**

In [ ]:
# feature scaling / normalization
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set,)

**Data Creation**

Using 30 values as input and using next 7 values as output for training.

E.g: Suppose our data set consists of 1 to 1000 numbers.

In the first step, the number 1 to 30 are inputs and the next 7 numbers are outputs (31,32,...37).

In the next step, the numbers 2 to 31 are inputs and 32,33,...,38 are outputs.

The model keeps on learning the future 7 values from the past 30 values.

In this way, we can predict the future values of temperature from the past values. 

In [ ]:
x_train = []
y_train = []

future = 7  # next 7 days temperature forecast
past = 30   # past 30 days 

for i in range(0,len(training_set_scaled)- past - future + 1):
    x_train.append(training_set_scaled[i : i + past , 0])     
    y_train.append(training_set_scaled[i + past : i + past + future , 0 ])
    
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1) )

**Model Creation and Training**

In [ ]:
regressor = Sequential()

regressor.add((Dense(units=30, input_shape = (x_train.shape[1],1) )))
regressor.add(Dense(units=30 , activation='relu'))
regressor.add(Dense(units=30 , activation='relu'))
regressor.add(Dense(units=30 , activation='relu'))
regressor.add(Flatten())
regressor.add(Dense(units = future))

regressor.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc'])

regressor.fit(x_train, y_train, epochs=100,batch_size=32 )

**Model Prediction and Accuracy Check**

In [ ]:
# read test dataset
testdataset = pd.read_csv('test_data.csv')

# get only the temperature column
testdataset = testdataset.iloc[:30,0:1].values

# get the actual next 7 outputs of temperature for comparison
real_temp = pd.read_csv('test_data.csv')
real_temp = real_temp.iloc[30:37,0:1].values

# normalization to make prediction
testing = sc.transform(testdataset)
testing = np.array(testing)
testing = np.reshape(testing,(testing.shape[1],testing.shape[0],1))

In [ ]:
# prediction and denormalization to get actual outputs
predicted_temperature = regressor.predict(testing)
predicted_temperature = sc.inverse_transform(predicted_temperature)
predicted_temperature = np.reshape(predicted_temperature,(predicted_temperature.shape[1],predicted_temperature.shape[0]))

In [ ]:
# real and predicted Values
print(real_temp, 'Real Temperature')
print(predicted_temperature, 'Predicted Temperature')

**Model Saving & Size Check**

In [ ]:
regressor.save("temperature_model.h5")
print ('TF Size:', round(os.path.getsize("temperature_model.h5")/(1024), 3) , 'KB')

# TF Lite Started

**TF Model to TF Lite Conversion**

In [ ]:
# convert & optimize the model
converter = tf.lite.TFLiteConverter.from_keras_model(regressor)
converter.optimization = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

**TF Lite Model Saving & Size Check**

In [ ]:
# save the model
open('tflite_model.tflite', 'wb').write(tflite_model)

# model size check
print ('TF Lite Size:', round(os.path.getsize('tflite_model.tflite')/(1024), 3) , 'KB')

**Interpreter**

In [ ]:
# 'interpreter' to interpret tf lite model and to get input & output details
# models obtained from 'TfLiteConverter' can be run in python with 'interpreter'
interpreter = tf.lite.Interpreter('tflite_model.tflite')

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# print input & output values & shapes to check everything is same 
# as the orignal tf model.
# data types etc are changed during tf lite conversion & optimizations
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

**Prediction from TF Lite Model**

In [ ]:
check = pd.read_csv('test_data.csv')
check = check.iloc[:30,0:1].values

# convert to new data type as orignal was float64
new_data = np.array(check, dtype=np.float32)

# reshaping data for input to tf lite model
testx = sc.transform(new_data)
testx = np.array(testx)
testx = np.reshape(testx,(testx.shape[1],testx.shape[0],1))

# needed before execution
# tensorFlow lite pre-plans tensor allocations to optimize inference,
# so the user needs to call allocate_tensors() before any inference.
interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'], testx)
interpreter.invoke()

# predicting from tf lite model
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])

# denormization and reshaping to show actual outputs
pt = sc.inverse_transform(tflite_model_predictions)
pt = np.reshape(pt,(pt.shape[1],pt.shape[0]))

print (real_temp, 'Real Temperature')
print (predicted_temperature, 'TF Predicted Temperature')
print (pt, 'TF Lite Predicted Temperature')


**Plotting Results for Comparison**

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(7, 5), dpi=150)
plt.plot(real_temp, color = 'black', label = 'Real Temperature')
plt.plot(predicted_temperature, color = 'green', label = 'TF Predicted Temperature')
plt.plot(pt, color = 'red', label = 'TF Lite Predicted Temperature')
plt.title('Temperature Comparison')
plt.xlabel('Days')
plt.ylabel('Temperature')
plt.legend()
plt.show()